# Agents in LlamaIndex

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

## Let's install the dependencies

We will install the dependencies for this unit.

In [5]:
!pip install llama-index llama-index-vector-stores-chroma llama-index-llms-groq llama-index-embeddings-huggingface -U -q

And, let's log in to Hugging Face to use serverless Inference APIs.

In [ ]:
from huggingface_hub import login

login()

## Initialising agents

Let's start by initialising an agent. We will use the basic `AgentWorkflow` class to create an agent.

In [6]:
from llama_index.llms.groq import Groq
from llama_index.core.agent.workflow import AgentWorkflow, ToolCallResult, AgentStream

from google.colab import userdata


def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers"""
    return a - b


def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


def divide(a: int, b: int) -> int:
    """Divide two numbers"""
    return a / b



llm = Groq(model="meta-llama/llama-4-maverick-17b-128e-instruct", api_key=userdata.get("GROQ_API_KEY"))

agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[subtract, multiply, divide, add],
    llm=llm,
    system_prompt="You are a math agent that can add, subtract, multiply, and divide numbers using provided tools.",
)

Then, we can run the agent and get the response and reasoning behind the tool calls.

In [7]:
handler = agent.run("What is (2 + 2) * 2?")
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing the thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp


Called tool:  add {'a': 2, 'b': 2} => 4

Called tool:  multiply {'a': 4, 'b': 2} => 8
The result of (2 + 2) * 2 is 8.

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='The result of (2 + 2) * 2 is 8.')]), structured_response=None, current_agent_name='Agent', raw={'id': 'chatcmpl-61929c09-ad04-4ee7-bd4b-8e9bcc70a44a', 'choices': [{'delta': {'content': None, 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None}], 'created': 1763828888, 'model': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'object': 'chat.completion.chunk', 'service_tier': None, 'system_fingerprint': 'fp_9b0c2006ef', 'usage': {'completion_tokens': 17, 'prompt_tokens': 1071, 'total_tokens': 1088, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.081391408, 'prompt_time': 0.060146825, 'completion_time': 0.01897505, 'total_time': 0.079121875}, 'x_groq': {'id': 'req_01kap69wnqf5kvphmvqvpkfzp2', 'usage': {'queue_time': 0.081391408, 'prompt_tok

In a similar fashion, we can pass state and context to the agent.


In [9]:
from llama_index.core.workflow import Context

ctx = Context(agent)

response = await agent.run("My name is Bob.", ctx=ctx)
response = await agent.run("What was my name again?", ctx=ctx)
response.response

ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Your name is Bob.')])

## Creating RAG Agents with QueryEngineTools

Let's now re-use the `QueryEngine` we defined in the [previous unit on tools](/tools.ipynb) and convert it into a `QueryEngineTool`. We will pass it to the `AgentWorkflow` class to create a RAG agent.

In [16]:
import chromadb

from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import QueryEngineTool
from llama_index.vector_stores.chroma import ChromaVectorStore

# Create a vector store
db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection("alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Create a query engine
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
llm = Groq(model="meta-llama/llama-4-maverick-17b-128e-instruct", api_key=userdata.get("GROQ_API_KEY"))

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model
)
query_engine = index.as_query_engine(llm=llm)
query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="personas",
    description="descriptions for various types of personas",
    return_direct=False,
)

# Create a RAG agent
query_engine_agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[query_engine_tool],
    llm=llm,
    system_prompt="You are a helpful assistant that has access to a database containing persona descriptions. ",
)

And, we can once more get the response and reasoning behind the tool calls.

In [17]:
handler = query_engine_agent.run(
    "Search the database for 'science fiction' and return some persona descriptions."
)
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing the thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp


Called tool:  personas {'input': 'science fiction'} => Empty Response
The search for 'science fiction' personas returned no results.

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text="The search for 'science fiction' personas returned no results.")]), structured_response=None, current_agent_name='Agent', raw={'id': 'chatcmpl-f99289e5-f508-4891-be1a-3a85ac80bab7', 'choices': [{'delta': {'content': None, 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None}], 'created': 1763829086, 'model': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'object': 'chat.completion.chunk', 'service_tier': None, 'system_fingerprint': 'fp_9b0c2006ef', 'usage': {'completion_tokens': 13, 'prompt_tokens': 797, 'total_tokens': 810, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.070109384, 'prompt_time': 0.01558298, 'completion_time': 0.038083918, 'total_time': 0.053666898}, 'x_groq': {'id': 'req_01kap6fy4ae74vek4xsafw8nyz', 'usage': {'queue_ti

## Creating multi-agent systems

We can also create multi-agent systems by passing multiple agents to the `AgentWorkflow` class.

In [18]:
from llama_index.core.agent.workflow import (
    AgentWorkflow,
    ReActAgent,
)


# Define some tools
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b


# Create agent configs
# NOTE: we can use FunctionAgent or ReActAgent here.
# FunctionAgent works for LLMs with a function calling API.
# ReActAgent works for any LLM.
calculator_agent = ReActAgent(
    name="calculator",
    description="Performs basic arithmetic operations",
    system_prompt="You are a calculator assistant. Use your tools for any math operation.",
    tools=[add, subtract],
    llm=llm,
)

query_agent = ReActAgent(
    name="info_lookup",
    description="Looks up information about XYZ",
    system_prompt="Use your tool to query a RAG system to answer information about XYZ",
    tools=[query_engine_tool],
    llm=llm,
)

# Create and run the workflow
agent = AgentWorkflow(agents=[calculator_agent, query_agent], root_agent="calculator")

# Run the system
handler = agent.run(user_msg="Can you add 5 and 3?")

In [19]:
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing the thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp

Thought: The current language of the user is: English. I need to use a tool to help me answer the question. The user is asking me to perform a simple arithmetic operation, which I can do using the "add" tool.
Action: add
Action Input: {"a": 5, "b": 3}
Called tool:  add {'a': 5, 'b': 3} => 8
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The sum of 5 and 3 is 8.

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='The sum of 5 and 3 is 8.')]), structured_response=None, current_agent_name='calculator', raw={'id': 'chatcmpl-855adfde-58e4-4e58-8343-5b2778092167', 'choices': [{'delta': {'content': None, 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None}], 'created': 1763829096, 'model': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'object': 'chat.completion.chunk', 'service_tier': None, 'system_fingerprint': 'fp_9b0c2006ef', 'usage': {'completion_tokens': 34, 'prompt_tokens': 812, 'total_tokens': 846, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.070400486, 'prompt_time': 0.016221408, 'completion_time': 0.348319517, 'total_time': 0.364540925}, 'x_groq': {'id': 'req_01kap6g7cyfy1vwmq9zhp247f0', 'usage': {'queue_time': 0.070400486, 'prompt_tokens